# I.Cài đặt thư viện

In [21]:
import torch
import os
import glob
from facenet_pytorch import InceptionResnetV1,fixed_image_standardization
from torchvision import transforms
import numpy as np
from PIL import Image

Chọn thiết bị để sau tính ma trận trên cái này

In [22]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# II. Chuẩn hóa
Để tránh hiện tượng Vanishing/ Exploding Gradient
Chuẩn hóa các giá trị về dạng [-1;1]

Hàm chuẩn hóa

In [23]:
def standardization(img_tensor_tmp):
    normalize = (img_tensor_tmp -127.5)/128
    return normalize

Tính toán trên toàn bộ tập dữ liệu

In [24]:
def trans(img_tmp):
    transform = transforms.Compose([
        transforms.ToTensor(),
        standardization
    ])
    return transform(img_tmp)

# III. Xây dựng mô hình

Sử dụng kiến trúc mạng InceptionResnetV1 đã được preTrain
- Đầu ra là feature vector 512 chiều

Chú thích
- classify: Có lấy kết quả đầu ra đã được classify theo tập pretrain không (nếu không thì đầu ra sẽ là feature vector)
- eval(): để khai báo mình đang evaluation chứ không phải trainning

In [25]:
model = InceptionResnetV1(
    classify=False,
    pretrained="casia-webface"
).to(device)

model.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

# IV. Tạo lập embed từ ảnh

Tạo địa đường dẫn thư mục

In [29]:
train_path = './data/data_set/train'
data_path = './data/annotations'

In [30]:
embeddings = []
names = []

In [31]:
for usr in os.listdir(train_path): # duyệt qua tất cả các file trong image_train
    embeds = []
    for file in glob.glob(os.path.join(train_path, usr) + '/*.jpg'): # mở tất cả các file ảnh
        try:
            img = Image.open(file)
        except:
            continue
        with torch.no_grad(): #Không đạo hàm ngược lại (cập nhập tham số) mà chỉ tính theo chiều xuôi
            embeds.append(model(trans(img).to(device).unsqueeze(0))) #thêm vào 1 chiều giả mạo kích thước [1,512]
    if len(embeds) == 0:
        continue
    else:
        embedding = torch.cat(embeds).mean(0, keepdim = True) # tinh gia tri trung binh cua 50 anh, giu nguyen chieu [1,512]
        embeddings.append(embedding) # them vao danh sach embeddings
        names.append(usr)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './data/data_set/train'

In [ ]:
embeddings = torch.cat(embeddings) # [n,512]
torch.save(embeddings, data_path + "/facelist.pth")
names = np.array(names)
np.save(data_path + "/username", names)

In [ ]:
print('cập nhập thành công ! Có {} người trong FaceLists'.format(names.shape[0]))